# 流程控制
|流程控制|符号|
|:---:|:---:|
|复合表达式|begin和（;）|
|条件表达式|if-elseif-else和?:(三元运算符)|
|短路求值|&&,&#124;&#124; 和链式结构|
|重复执行|循环：while和for|
|异常处理|try-catch,error和throw|
|Task（协程）|yieldto|
* Task 不是那么的标准：它提供了非局部的流程控制，这使得在暂时挂起的计算任务之间进行切换成为可能。这是一个功能强大的构件：Julia 中的异常处理和协同多任务都是通过 Task 实现的。虽然日常编程并不需要直接使用 Task，但某些问题用 Task 处理会更加简单。
## 复合表达式
有时一个表达式能够有序地计算若干子表达式，并返回最后一个子表达式的值作为它的值是很方便的。Julia 有两个组件来完成这个： begin 代码块 和 (;) 链。
## 条件表达式
if 代码块是"有渗漏的"，也就是说它们不会引入局部作用域。这意味着在 if 语句中新定义的变量依然可以在 if 代码块之后使用，尽管这些变量没有在 if 语句之前定义过。<br/>

与 C, MATLAB, Perl, Python，以及 Ruby 不同，但跟 Java，还有一些别的严谨的类型语言类似：**一个条件表达式的值如果不是 true 或者 false 的话，会返回错误**<br/>

```Julia
if 1 
    println("true")
end
```
会因为1是Int64而报错。<br/>

所谓的 "三元运算符", ?:，很类似 if-elseif-else 语法，它用于选择性获取单个表达式的值，而不是选择性执行大段的代码块。它因在很多语言中是唯一一个有三个操作数的运算符而得名：<br/>
```Julia
a ? b : c
```

在 ? 之前的表达式 a, 是一个条件表达式，**如果条件 a 是 true，三元运算符计算在 : 之前的表达式 b；如果条件 a 是 false，则执行 : 后面的表达式 c。** ***注意，? 和 : 旁边的空格是强制的***<br/>
## 短路求值
短路求值非常类似条件求值。这种行为在多数有&&和 ||布尔运算符地命令式编程语言里都可以找到：在一系列由这些运算符连接的布尔表达式中，为了得到整个链的最终布尔值，仅仅只有最小数量的表达式被计算：<br/>
* 在表达式a&&b中，子表达式b仅当a为真时才执行；
* 在表达式a||b中，子表达式b仅当a为假时才会执行；
  ** 如果 a 是 false，那么无论 b 的值是多少，a && b 一定是 false。同理，如果 a 是 true，那么无论 b 的值是多少，a || b 的值一定是 true。&&和||都依赖于右边，但是&&比||有更高的优先级。

## 循环
for 循环与之前 while 循环的一个非常重要区别是作用域，即变量的可见性。如果变量 i 没有在另一个作用域里引入，在 for 循环内，它就只在 for 循环内部可见，在外部和后面均不可见。你需要一个新的交互式会话实例或者一个新的变量名来测试这个特性<br/>
一般来说，for 循环组件可以用于迭代任一个容器。在这种情况下，相比 =，另外的（但完全相同）**关键字 in 或者 ∈ 则更常用，因为它使得代码更清晰**<br/>
我们可能会在测试条件不成立之前终止一个 while 循环，或者在访问到迭代对象的结尾之前停止一个 for 循环，这可以用关键字 break 来完成<br/>
## 异常处理
当一个意外条件发生时，一个函数可能无法向调用者返回一个合理的值。在这种情况下，最好让意外条件终止程序并打印出调试的错误信息，或者根据程序员预先提供的异常处理代码来采取恰当的措施。<br/>
你可能需要根据下面的方式来定义你自己的异常：<br/>

```Julia 
struct MyCustomException<:Exception end

```
### 错误
我们可以用 error 函数生成一个 ErrorException 来中断正常的控制流程。<br/>
### finally子句
在进行状态改变或者使用类似文件的资源的编程时，经常需要在代码结束的时候进行必要的清理工作（比如关闭文件）。由于异常会使得部分代码块在正常结束之前退出，所以可能会让上述工作变得复杂。finally 关键字提供了一种方式，无论代码块是如何退出的，都能够让代码块在退出时运行某段代码。<br/>

## Task（协程）

Task 是一种允许计算以更灵活的方式被中断或者恢复的流程控制特性。这个特性有时被叫做其它名字，例如，对称协程（symmetric coroutines），轻量级线程（lightweight threads），合作多任务处理（cooperative multitasking），或者单次续延（one-shot continuations）。<br/>
当一部分计算任务（在实际中，执行一个特定的函数）可以被设计成一个 Task 时，就可以中断它，并切换到另一个 Task。原本的 Task 可以恢复到它上次中断的地方，并继续执行。第一眼感觉，这个跟函数调用很类似。但是有两个关键的区别。首先，是切换 Task 并不使用任何空间，所以任意数量的 Task 切换都不会使用调用栈（call stack）。其次，Task 可以以任意次序切换，而不像函数调用那样，被调用函数必须在返回主调用函数之前结束执行。<br/>
这种流程控制的方式使得解决一个特定问题更简便。在一些问题中，多个需求并不是有函数调用来自然连接的；在需要完成的工作之间并没有明确的“调用者”或者“被调用者”。一个例子是生产-消费问题，一个复杂的流程产生数据，另一个复杂的流程消费他们。消费者不能简单的调用生产函数来获得一个值，因为生产者可能有更多的值需要创建，还没有准备好返回。用 Task 的话，生产者和消费者能同时运行他们所需要的任意时间，根据需要传递值回来或者过去。<br/>

Julia 提供了** Channel 机制来解决这个问题。一个 Channel 是一个先进先出的队列，允许多个 Task 对它可以进行读和写**。<br/>
注意我们并不需要显式地在生产者中关闭 Channel。这是因为 Channel 对 Task 的绑定同时也意味着 Channel 的生命周期与绑定的 Task 一致。当 Task 结束时，Channel 对象会自动关闭。多个 Channel 可以绑定到一个 Task，反之亦然。

尽管 Task 的构造函数只能接受一个“无参函数”，但 Channel 方法会创建一个与 Channel 绑定的 Task，并令其可以接受 Channel 类型的单参数函数。一个通用模式是对生产者参数化，此时需要一个部分函数应用来创建一个无参，或者单参的匿名函数。

对于 Task 对象，可以直接用，也可以为了方便用宏。
```Julia
function mytask(myarg)
    ...
end
taskHdl=Task(()->mytask(7))
#or,equivalently
taskHdl=@task mytask(7)
```
为了安排更高级的工作分配模式，bind 和 schedule 可以与 Task 和 Channel 构造函数配合使用，显式地连接一些 Channel 和生产者或消费者 Task。

注意目前 Julia 的 Task 并不分配到或者运行在不同的 CPU 核心上。真正的内核进程将在分布式计算进行讨论。

### Task相关的核心操作
让我们来学习底层构造函数 yieldto 来理解 Task 是如何切换工作的。**yieldto(task,value) 会中断当前的 Task，并切换到特定的 Task，并且 Task 的最后一次 yieldto 调用会有特定的返回值。注意 yieldto 是唯一一个需要用任务类型的流程控制的操作，仅需要切换到不同的 Task，而不需要调用或者返回。这也就是为什么这个特性会被叫做“对称协程（symmetric coroutines）”；每一个 Task 以相同的机制进行切换或者被切换。**

yieldto 功能强大，但大多数 Task 的使用都不会直接调用它。思考为什么会这样。如果你切换当前 Task，你很可能会在某个时候想切换回来。但知道什么时候切换回来和哪个 Task 负责切换回来需要大量的协调。例如，**put! 和 take! 是阻塞操作**，当在渠道环境中使用时，维持状态以记住消费者是谁。不需要人为地记录消费 Task，正是使得 put! 比底层 yieldto 易用的原因。

除了 yieldto 之外，也需要一些其它的基本函数来更高效地使用 Task。

| | |
|:-----:|:-------:|
|current_task |获取当前运行 Task 的索引。|
|istaskdone |查询一个 Task 是否退出|
|istaskstarted |查询一个 Task 是否已经开始运行。|
|task_local_storage |操纵针对当前 Task 的键值存储。|
### Task 和事件
多数 Task 切换是在等待如 I/O 请求的事件，由 Julia Base 里的调度器执行。调度器维持一个可运行 Task 的队列，并执行一个事件循环，来根据例如收到消息等外部事件来重启 Task。

**等待一个事件的基本函数是 wait。**很多对象都实现了 wait 函数；例如，给定一个 Process 对象，wait 将等待它退出。wait 通常是隐式的，例如，wait 可能发生在调用 read 时等待数据可用。

在所有这些情况下，**wait 最终会操作一个 Condition 对象，由它负责排队和重启 Task。当 Task 在一个 Condition 上调用 wait 时，该 Task 就被标记为不可执行，加到条件的队列中，并切回调度器。调度器将选择另一个 Task 来运行，或者阻止外部事件的等待。如果所有运行良好，最终一个事件处理器将在这个条件下调用 notify，使得等待该条件的 Task 又变成可运行。**

调用 Task 显式创建的 Task 对于调度器时来说一开始时不知道的。如果你希望的话，你可以使用 yieldto 来人为管理 Task。但是当这种 Task 等待一个事件时，正如期待的那样，当事件发生时，它将自动重启。也能由调度器在任何可能的时候运行一个 Task，而无需等待任何事件。这可以调用 schedule，或者使用 @async 宏（见并行计算中的详细说明）。
### Task的状态
Task 由 state 属性来描述他们的执行状态。Task state 有：

|符号|	含义|
|:---:|:-----:|
|:runnable	|正在运行，或者可以被切换到|
|:waiting	|被阻塞，等待一个特定事件|
|:queued	|处在调度器中的运行队列中，即将被重启|
|:done	|成功结束执行|
|:failed	|以一个没被捕获的异常结束|

In [2]:
#####复合表达式##############################################
z=begin#有时一个表达式能够有序地计算若干子表达式，
    #并返回最后一个子表达式的值作为它的值是很方便的。
    x=1
    y=2
    x+y
end

z=(x=1;y=2;x+y)#它们可以简单地被放到一行里，这也是 (;) 链的由来

begin x=1;y=2;x+y end#并不要求 begin 代码块是多行的
(x=1;y=2;x+y)#或者 (;) 链是单行的

####条件表达式################################################
if x<y
    println("x is less than y")
elseif x>y
    println("x>y")
else
    println("x=y")
end

#与 C, MATLAB, Perl, Python，Ruby 不同，但跟 Java等严谨的类型语言类似：
#一个条件表达式的值如果不是 true 或者 false 的话，会返回错误

#所谓的 "三元运算符", ?:，很类似 if-elseif-else 语法，
#它用于选择性获取单个表达式的值，而不是选择性执行大段的代码块。
#它因在很多语言中是唯一一个有三个操作数的运算符而得名：

x=1;y=2;
println(x<y ? "less than" : "not less than")

###三元运算符的链式嵌套####
(x,y)=println(x<y ? "x is less than y" : 
x>y ? "x is greater than y" : "x is equal to y")

#与 if-elseif-else 类似，
#: 之前和之后的表达式只有在条件表达式为 true 或者 false 时才会被相应地执行
v(x)=(println(x);x)
1<2 ? v("yes") : v("no")
1>2 ? v("yes") : v("no")




x is less than y
less than


In [4]:
####短路求值###########################################
t(x)=(println(x);true)
f(x)=(println(x);false)
t(1)&&t(2)#1;2;true

t(1)&&f(2)#1;2;false

f(1)&&t(2)#1;false#a为假只执行第二句

f(1)&&f(2)#1;false

t(1)||t(2)#1;true#a为真只执行第二句

t(1)||f(2)#1;true

f(1)||t(2)#1;2;true

f(1)||f(2)#1;2;false

#######
function fact(n::Int)
    n>=0 || error("n must be non-negative")# if 
    n==0 && return 1#if !
    n*fact(n-1)
end
#fact(-1)
fact(4)
 
#无短路求值的运算也可以用位布尔运算符完成，见数学运算及初等函数：&，|
f(1) & t(2)
t(1) | t(2)
#与if elseif 或者三元运算符中的条件表达式相同，&&或||的操作必须是布尔值,除最后一项

#1 && true 会报错
#但在链的末尾允许使用任意类型的表达式，此表达式会根据前面的条件被执行并返回
true && (x=(1,2,3))



1
2
1
2
1
1
1
1
1
2
1
2


24

In [9]:
#############################循环############################################
i=1
while i<=5
    println(i)
    global i+=1
end
for i∈1:5 
    println(i)
end
# \=,in
i=1


#我们可能会在测试条件不成立之前终止一个 while 循环，
#或者在访问到迭代对象的结尾之前停止一个 for 循环，这可以用关键字 break 来完成

while true
    println(i)
    if i>=5
        break
    end
    global i+=1
end

for j∈1:1000
    println(j)
    if j>=5
        break
    end
end

#需要直接结束此次迭代，并立刻进入下次迭代，continue 关键字
for i=1:10
    if i%3!=0
        continue
    end
    println(i)
end
#在实际应用中，在 continue 后面还会有更多的代码要运行，
#并且调用 continue 的地方可能会有多个

#多个嵌套的 for 循环可以合并到一个外部循环，可以用来创建其迭代对象的笛卡尔积
for i=1:2,j=3:5
    println((i,j))
end
println("###################i#############")
for i=1:5,j=3:i
    println((i,j))#但是在一个循环里面使用 break 语句则会跳出整个嵌套循环，
    #不仅仅是内层循环。
end


1
2
3
4
5
1
2
3
4
5
1
2
3
4
5
1
2
3
4
5
3
6
9
(1, 3)
(1, 4)
(1, 5)
(2, 3)
(2, 4)
(2, 5)
###################i#############
(3, 3)
(4, 3)
(4, 4)
(5, 3)
(5, 4)
(5, 5)


In [19]:
#######################异常处理##############################
#我们可以用 throw 显式地创建异常
f(x)=x>=0 ? exp(-x) : throw(DomainError(x,"argument must be nonnegative"))
#f(-1)

#DomainError后面不接括号的话不是一个异常而是一个异常类型，我们需要调用
#它来获得一个Exception对象。
typeof(DomainError(nothing))<:Exception
typeof(DomainError)<:Exception

#throw(UndefVarError(:x))

struct MyUndefVarError<:Exception
    var::Symbol
end

#错误信息的第一个单词最好用小写，除非相关的参数是大写的。
#############错误#################

fussy_sqrt(x)=x>=0 ? sqrt(x) : error("negative x not allowed ")

function verbose_fussy_sqrt(x)
    println("before fussy_sqrt")
    r=fussy_sqrt(x)#若产生错误，则后续不会执行
    println("after fussy_sqrt")
    return r
end

###########try-catch###########异常处理保证程序遇到错误仍能正常进行
f(x)=try
    sqrt(x)
catch #在实际用这个函数的时候，应该比较 x 与 0 的大小，
    #而不是捕获一个异常，异常比直接使用判断分支慢得多。
    sqrt(complex(x,0))
end


#try/catch 语句允许保存 Exception 到一个变量中。
sqrt_second(x)=
try
    sqrt(x[2])
    catch y# catch 后面的字符会被一直认为是异常的名字
        if isa(y,DomainError)
            sqrt(complex(x[2],0))
        elseif isa(y,BoundsError)
            sqrt(x)
    end
end

sqrt_second([1,-4])

#try bad() catch x end,不会返回x
#try bad() catch; x end,返回x

#try/catch 组件的强大之处在于能够将高度嵌套的计算立刻解耦成更高层次地调用函数。有时没有错误产生，但需要能够解耦堆栈，并传值到上层。
#Julia 提供了 rethrow, backtrace 和 catch_backtrace 函数进行更高级的错误处理。

#finally子句
f=open("file")
try
    #operate on file f
finally
    close(f)
end
#当控制流离开 try 代码块（遇到 return，或者正常结束），close(f) 就会被执行。
#如果 try 代码块由于异常退出，这个异常会继续传递。
#catch 代码块可以和 try 还有 finally 配合使用。
#这时 finally 代码块会在 catch 处理错误之后才运行。



0.0 + 2.0im

In [25]:
##############Task(协程)#####################
#使用一个特殊的 Channel 组件来运行一个与其绑定的 Task，
#它能接受单参数函数作为其参数，然后可以用 take! 从 Channel 对象里不断地提取值
function producer(c::Channel)
    put!(c,"start")
    for n=1:4
        put!(c,2n)
    end
    put!(c,"stop")
end
chnl=Channel(producer);
take!(chnl)
take!(chnl)
take!(chnl)
take!(chnl)
take!(chnl)
take!(chnl)
#一种思考这种行为的方式是，“生产者”能够多次返回。
#在两次调用 put! 之间，生产者的执行是挂起的，此时由消费者接管控制。

#返回的 Channel 可以被用作一个 for 循环的迭代对象，
#此时循环变量会依次取到所有产生的值。当 Channel 关闭时，循环就会终止。
for x in Channel(producer)
    println(x)
end
#我们并不需要显式地在生产者中关闭 Channel。
#因为 Channel 对 Task 的绑定同时也意味着 Channel 的生命周期与绑定的 Task 一致。
#当 Task 结束时，Channel 对象会自动关闭。
#多个 Channel 可以绑定到一个 Task，反之亦然。

#尽管 Task 的构造函数只能接受一个“无参函数”，
#但 Channel 方法会创建一个与 Channel 绑定的 Task，
#并令其可以接受 Channel 类型的单参数函数。一个通用模式是对生产者参数化，
#此时需要一个部分函数应用来创建一个无参，或者单参的匿名函数。

#对于 Task 对象，可以直接用，也可以为了方便用宏。

start
2
4
6
8
stop
